In [1]:
import requests
import pandas as pd

API_KEY = "71cbe35ff6e0c0df92b72d8a233c40a85de5b0b9"

url = (
    "https://api.census.gov/data/2023/acs/acs1/pums?"
    "get=AGEP,SEX,SCHL,OCCP,INDP,PINCP,RAC3P,WKHP,ESR,STATE"
    f"&for=state:*&key={API_KEY}"
)

# Fetch Data
response = requests.get(url)
print("Status:", response.status_code)
print("Content-Type:", response.headers.get("Content-Type"))

if response.status_code != 200:
    print("Error:", response.text[:500])
    raise SystemExit

data = response.json()

# Convert JSON → DataFrame
columns = data[0]
rows = data[1:]
df = pd.DataFrame(rows, columns=columns)

# Clean Column Names
df = df.rename(columns={
    "AGEP": "age",
    "SEX": "gender_code",
    "SCHL": "educational_attainment",
    "OCCP": "occupation_code",
    "INDP": "industry_code",
    "PINCP": "income",
    "RAC3P": "race_code",
    "WKHP": "hours_per_week",
    "ESR": "employment_status_code",
    "STATE": "state_code"
})

# Convert Numeric Columns
num_cols = [
    "age", "gender_code", "educational_attainment", "occupation_code",
    "industry_code", "income", "race_code", "hours_per_week",
    "employment_status_code", "state_code"
]
for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Add Readable Mappings
df["gender"] = df["gender_code"].map({1: "Male", 2: "Female"})
df["employment_status"] = df["employment_status_code"].map({
    1: "Civilian, employed, at work",
    2: "Civilian, employed, with a job but not at work",
    3: "Unemployed",
    4: "Armed forces, at work",
    5: "Armed forces, with a job but not at work",
    6: "Not in labor force"
})

Status: 200
Content-Type: application/json;charset=utf-8


In [2]:
# Display Preview
print("Total rows:", len(df))
df.head(10)

Total rows: 3405809


,age,gender_code,educational_attainment,occupation_code,industry_code,income,race_code,hours_per_week,employment_status_code,state_code,state,gender,employment_status
0,94,2,16,NaN,NaN,12200,1,0,6,34,34,Female,Not in labor force
1,18,1,19,NaN,NaN,0,1,0,6,34,34,Male,Not in labor force
2,75,1,21,NaN,NaN,108000,29,0,6,8,08,Male,Not in labor force
3,44,1,18,NaN,NaN,0,2,0,6,12,12,Male,Not in labor force
4,31,1,18,4220.0,7580.0,5200,2,20,6,39,39,Male,Not in labor force
5,19,2,19,NaN,NaN,0,10,0,6,36,36,Female,Not in labor force
6,36,1,16,9130.0,6170.0,0,1,0,6,55,55,Male,Not in labor force
7,47,2,15,NaN,NaN,15300,1,0,6,13,13,Female,Not in labor force
8,19,2,19,4055.0,8680.0,3600,15,16,1,6,06,Female,"Civilian, employed, at work"
9,15,1,12,NaN,NaN,0,2,0,0,12,12,Male,NaN


In [3]:
df.to_parquet("data.parquet", index=False)